# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [2]:
from openai import OpenAI

client = OpenAI(base_url="https://api.xiaoai.plus/v1")

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这张图片展示了一个宁静的自然景观，画面中心是一条铺设整齐的木板路，引领我们的视线深入到远处的天际线。周围是茂盛的绿色草地，点缀着一些低矮的灌木和树木。天空是晴朗的，有几朵轻盈的白云。图片整体呈现出平和与自然美的氛围，是一个适合散步和放松的地方。光线柔和，色彩鲜明，给人一种清新和宁静的感觉。', role='assistant', function_call=None, tool_calls=None))


In [3]:
response.choices[0].message.content

'这张图片展示了一个宁静的自然景观，画面中心是一条铺设整齐的木板路，引领我们的视线深入到远处的天际线。周围是茂盛的绿色草地，点缀着一些低矮的灌木和树木。天空是晴朗的，有几朵轻盈的白云。图片整体呈现出平和与自然美的氛围，是一个适合散步和放松的地方。光线柔和，色彩鲜明，给人一种清新和宁静的感觉。'

### 封装成一个函数 query_image_description

In [4]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(base_url="https://api.xiaoai.plus/v1")  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [5]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图中展示了两只狗的形象，一只被搞笑地改造成有着健美肌肉的外观，另一只是较为常见的柴犬坐姿。图中的文字有点像搞笑对比，左边的狗被形容为“16岁的我，工作后的我”，拥有壮硕的身体和理想化的特征，而右边的狗则表现出了一种较为消极或现实的状态，与理想化的自我形象形成对比。整体而言，这幅图具有幽默和讽刺的风格，通过夸张的对比来表达生活中理想与现实之间的差距。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [9]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(base_url="https://api.xiaoai.plus/v1")  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [10]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图展示了从1980年到2020年期间，美国、中国、日本和德国的国内生产总值（GDP）的比较。图中用不同颜色的线表示不同国家的GDP走势：

- 蓝色线代表美国的GDP，从1980年约1万亿美元逐步增加至2020年接近22万亿美元。
- 红色线代表中国的GDP，从1980年的低起点明显上升，2000年后增速加快，2020年接近15万亿美元。
- 紫色线代表日本，GDP在1995年左右达到峰值后基本保持平稳，2020年在5万亿美元左右。
- 绿色线代表德国，GDP增长相对平稳，从1980年的不到1万亿美元，增长至2020年的4万亿美元左右。

整体上，美国的GDP在整个时间段内保持全球领先，中国的GDP增长非常迅速，尤其是进入21世纪后，增长速度显著增加。日本的GDP在20世纪90年代达到顶峰之后，增长放缓。德国的GDP增长较为稳定。这张图很好地反映了这四个经济体在过去40年的经济发展轨迹。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [11]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片是一本笔记本的一页，上面记录了一些关于机器学习模型调整的笔记，特别是与自然语言处理相关的内容。以下是图片内容的详细解释：

1. **标题“浅谈简单的PETF方式”** - 这意味着笔记的焦点是简单的深度学习模型微调技术。

2. **Prompt Tuning (PMT & Small Model)** - 提到了Prompt Tuning，这是一种用于调整预训练模型的方法，通过微调少量的参数（例如在小模型中）来适应特定任务。
   - 笔记指出，X表示一个向量或一系列输入，Y表示输出结果。

3. **Prefix Tuning** - 跟Prompt Tuning类似，但专注于模型输入的前缀，尝试通过调整前缀来影响模型的输出行为。

4. 笔记中还提到了**LoRA (Low-Rank Adaptation)**技术，这是一种通过低秩矩阵增量更新原有权重来微调神经网络的策略。这里展示了如何通过修改权重矩阵W，增加一个微小的ΔW来达到调整的目的。与之相关的是两个公式:
   - `\Delta W = AB` - 这表明ΔW是两个矩阵A和B的乘积。
   - 改进后权重的更新可以表示为 `Y = (W + \Delta W)X`.

5. 提到了具体的例子，如**QLoRA**，还对比了两种调整策略使用的资源（比如内存），显示**LoRA**需要78GB，而**QLoRA**只需48GB。

这些笔记为深入了解如何通过不同的技术来优化和调整深度学习模型提供了一个基础概述。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [12]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片是一本笔记本的一页，上面记录了一些关于机器学习模型调整的笔记，特别是与自然语言处理相关的内容。以下是图片内容的详细解释：

1. **标题“浅谈简单的PETF方式”** - 这意味着笔记的焦点是简单的深度学习模型微调技术。

2. **Prompt Tuning (PMT & Small Model)** - 提到了Prompt Tuning，这是一种用于调整预训练模型的方法，通过微调少量的参数（例如在小模型中）来适应特定任务。
   - 笔记指出，X表示一个向量或一系列输入，Y表示输出结果。

3. **Prefix Tuning** - 跟Prompt Tuning类似，但专注于模型输入的前缀，尝试通过调整前缀来影响模型的输出行为。

4. 笔记中还提到了**LoRA (Low-Rank Adaptation)**技术，这是一种通过低秩矩阵增量更新原有权重来微调神经网络的策略。这里展示了如何通过修改权重矩阵W，增加一个微小的ΔW来达到调整的目的。与之相关的是两个公式:
   - `\Delta W = AB` - 这表明ΔW是两个矩阵A和B的乘积。
   - 改进后权重的更新可以表示为 `Y = (W + \Delta W)X`.

5. 提到了具体的例子，如**QLoRA**，还对比了两种调整策略使用的资源（比如内存），显示**LoRA**需要78GB，而**QLoRA**只需48GB。

这些笔记为深入了解如何通过不同的技术来优化和调整深度学习模型提供了一个基础概述。

In [ ]:
![](./images/handwriting_1.jpg)

In [13]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示了一些笔记内容，主要关注的是深度学习中的变换器（Transformer）模型及其不同的调整方法。内容清楚地分成两栏，使用中文和英文双语。

左栏内容包括：
- “转换模型”和“基准测试”（Benchmark）的研究。
- 提到了PEFT和SOTA（State of the Art）。
- 不同的方法包括PBFT Methods。
- 探讨了不同的prompt tuning技术，包括Adapter（2019, Google），Prefix（2021, Stanford），以及其他的prompt技术。
- 引入了一些与prompt tuning相关的改进方法，如soft prompts和 hand prompts。
- 还提及了Instruction tuning和一些具体的模型如LaMDA, GPT-4, CHATGLM, BLOOM, Alpaca。

右栏内容包括：
- 讨论了多模态指令微调技术，如Llama (LLaMA)。
- 提到了新的方法如LoRA和PETC。
- 详细讨论了prefix tuning和adapters的应用。
- “Project down - zero embed - project up”这句话可能描述了某种网络结构调整或数据处理方式。
- 最后列出了一些特定的方法绑定和他们的效果，如MAM Adapters。

总得来说，这些笔记围绕最新的AI技术发展，尤其是在模型微调和参数调整方法上的探索。这可能是一个科研人员或技术工作者在学习和总结相关最新知识点的记录。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。


### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [16]:
content = query_base64_image_description("./images/gpt-4v.jpg")
display(Markdown(content))

这张图片显示了一段手写的中文笔记，标题是“Long Chain Agents 连锁反应”，下方有两条列出的内容：

1. 结链在纸面中。一条线性链或分子链。
2. 在化学中，连接起来的其他分子或基团。

这些内容似乎是关于化学或分子结构的笔记，提到了链式反应和分子链的概念。